In [ ]:
import pandas as pd
import sqlite3

# CSV dosyasını oku
df = pd.read_csv('application_train.csv')

# SQLite veritabanına bağlan (yoksa oluşturur)
conn = sqlite3.connect('home_credit_db.sqlite')

# Veriyi SQL tablosuna yaz
df.to_sql('train_data', conn, if_exists='replace', index=False)

print("Veri başarıyla SQL ortamına yüklendi! 🚀")

In [ ]:
%load_ext sql
%sql sqlite:///home_credit_db.sqlite

In [ ]:
%%sql
SELECT
    COUNT(*) AS toplam_basvuru_sayisi,
    SUM(TARGET) AS temerrude_dusen_sayisi,
    ROUND(AVG(TARGET) * 100, 2) AS temerrut_orani_yuzde
FROM
    train_data;